# 1. Task 1

### 1.1. Overview

Model performance results for Blue group: 

In [1003]:
task1_display_results(task1_blue_results, "Blue")

array([['Blue', 'Will use XAI', 'Will not use XAI', 'Total'],
       ['Enrolled in training', '60', '5', '65'],
       ['not enrolled in training', '20', '15', '35'],
       ['Total', '80', '20', '100']], dtype='<U24')

We have a rule stating when to enroll somebody from the red population, instead of an example of an actual assignment. We can easily proof that this information can be presented in the table of probabilities below.

(one note: blue population is 9 times larger, although 100 doesn't divide by 9, hence I supposed it was meant to be 10 times larger):

In [1004]:
task1_display_results(task1_red_results, "Red")

array([['Red', 'Will use XAI', 'Will not use XAI', 'Total'],
       ['Enrolled in training', '0.25', '0.25', '0.5'],
       ['not enrolled in training', '0.25', '0.25', '0.5'],
       ['Total', '0.5', '0.5', '1.0']], dtype='<U32')

### 1.2. Calculate demographic parity

If model is meant to pass demographic parity check, results must satisfy the formula $$P(Enrolled=1\;|\;Group=blue)\;=\;P(Enrolled=1\;|\;Group=red)$$
In practice (legally - "four-fifth rule") it suffices, if $$0.8 \leq \frac{P(Enrolled=1\;|\;Group=blue)}{P(Enrolled=1\;|\;Group=red)} \leq 1.25$$
For blue and red groups those probabilities are:

In [1005]:
show_demographic_parity("blue", "red")

Demographic parity:
	blue: 0.35
	red: 0.5
	ratio: 0.7


### 1.3 Calculate equal opportunity

If model is meant to pass demographic parity check, results must satisfy the formula $$P(Enrolled=1\;|\;Group=blue\;\land\;WouldUse=1)\;=\;P(Enrolled=1\;|\;Group=red\;\land\;WouldUse=1)$$
Again, we would want to check appropriately expressed "four-fifth rule". For blue and red groups those probabilities are:

In [1006]:
show_equal_opportunity("blue", "red")

Equal opportunity:
	blue: 0.75
	red: 0.5
	ratio: 1.5


### 1.4 Calculate predictive rate parity
If model is meant to pass predictive rate parity check, results must satisfy formulas for equal positive/negative predictive value: $$P(WouldUse=1\;|\;Group=blue\;\land\;Enrolled=1)\;=\;P(WouldUse=1\;|\;Group=red\;\land\;Enrolled=1)$$ $$P(WouldUse=1\;|\;Group=blue\;\land\;Enrolled=0)\;=\;P(WouldUse=1\;|\;Group=red\;\land\;Enrolled=0)$$
Again, we would want to check appropriately expressed "four-fifth rule". For blue and red groups probabilities for positive/negative predictive value are:

In [1007]:
show_predictive_rate_parity("blue", "red")

Positive Predictive Parity:
	blue: 0.92
	red: 0.5
	ratio: 1.84
Negative Predictive Parity:
	blue: 0.43
	red: 0.5
	ratio: 0.86


### 1.5 Summary
As we can see, ratios were always beyond acceptable range, so proposed fairness metrics proved this model to be unfair. We could try to improve that by choosing more accurate rule to enroll people from red population to decrease amount of false-positives and false-negatives.

# 2. Task 2

### 2.1 Overview 
I have decided to choose liner regression models and the adult_income dataset. Two models will be trained with different hyperparamenters on dissimilarly prepared data using train/test split. One of them will be then enhanced using adversarial training technique and compared to the rest in terms of fairness and performance.

### 2.1 Train a model for the selected dataset.
First of all, we load the data:

In [1008]:
df.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K


We may notice that provided non-numeric values need to be transformed into (optionally standardised) numerical form (this also applies to undefined values marked here as '?'). We can do this in many ways, in particular, for the first model we are going to assign an ordering number for each distinct value (including '?') in every column. Regarding the second one, we will do the same, but also with additional step of standardisation.

Let's check the number of distinct values in each column:

In [1009]:
check_unique_input_values()

age                   74
workclass              9
fnlwgt             28523
education             16
educational-num       16
marital-status         7
occupation            15
relationship           6
race                   5
gender                 2
capital-gain         123
capital-loss          99
hours-per-week        96
native-country        42
income                 2
dtype: int64

During data preparation we encounter on 'income' column. Those values seem to be cumbersome in terms of transformations, but it turns out, this column contains only the information whether the income is greater or lesser than fixed 50K. Those can be replaced by binary values - which are going to be guessed by models.

We transform the data as described above and standardise them (for the second model only). A little glimpse on the result:

In [1010]:
X1.head()

,workclass,education,marital-status,occupation,relationship,race,gender,native-country,age,fnlwgt,educational-num,capital-gain,capital-loss,hours-per-week
0,4.0,1.0,4.0,7.0,3.0,2.0,1.0,39.0,25.0,226802.0,7.0,0.0,0.0,40.0
1,4.0,11.0,2.0,5.0,0.0,4.0,1.0,39.0,38.0,89814.0,9.0,0.0,0.0,50.0
2,2.0,7.0,2.0,11.0,0.0,4.0,1.0,39.0,28.0,336951.0,12.0,0.0,0.0,40.0
3,4.0,15.0,2.0,7.0,0.0,2.0,1.0,39.0,44.0,160323.0,10.0,7688.0,0.0,40.0
4,0.0,15.0,4.0,0.0,3.0,4.0,0.0,39.0,18.0,103497.0,10.0,0.0,0.0,30.0


In [1011]:
X2.head()

,workclass,education,marital-status,occupation,relationship,race,gender,native-country,age,fnlwgt,educational-num,capital-gain,capital-loss,hours-per-week
0,0.088484,-2.397350,0.916138,0.099824,0.971649,-1.971746,0.70422,0.289462,-0.995129,0.351675,-1.197259,-0.144804,-0.217127,-0.034087
1,0.088484,0.183660,-0.410397,-0.372938,-0.900852,0.392384,0.70422,0.289462,-0.046942,-0.945524,-0.419335,-0.144804,-0.217127,0.772930
2,-1.277432,-0.848744,-0.410397,1.045346,-0.900852,0.392384,0.70422,0.289462,-0.776316,1.394723,0.747550,-0.144804,-0.217127,-0.034087
3,0.088484,1.216063,-0.410397,0.099824,-0.900852,-1.971746,0.70422,0.289462,0.390683,-0.277844,-0.030373,0.886874,-0.217127,-0.034087
4,-2.643348,1.216063,0.916138,-1.554840,0.971649,0.392384,-1.42001,0.289462,-1.505691,-0.815954,-0.030373,-0.144804,-0.217127,-0.841104


At the very end of preparations, we divide our prepared data onto train and test sets (train/test split technique) and then we train two linear regression models.

### 2.2. For the selected protected attribute (age, gender, race) calculate the following fairness coefficients: Statistical parity, Equal opportunity, Predictive parity.

First, we compute classification matrices (consisting of TP, TN, FN, FP) for each value of protected attribute (I chose gender). It will allow us to compute fairness metrics as in the task 1

In [1012]:
cm1 = calculate_classification_matrix(X1_test, y1_test, model1_pred, "gender")
cm2 = calculate_classification_matrix(X2_test, y2_test, model2_pred, "gender")

##### 2.2.1 Statistical Parity (Demographic Parity):

In [1013]:
show_demographic_parity("Model 1 - Male", "Model 1 - Female", cm1[0], cm1[1])

Demographic parity:
	Model 1 - Male: 0.09
	Model 1 - Female: 0.22
	ratio: 0.41


In [1014]:
show_demographic_parity("Model 2 - Male", "Model 2 - Female", cm2[0], cm2[1])

Demographic parity:
	Model 2 - Male: 0.28
	Model 2 - Female: 0.13
	ratio: 2.15


##### 2.2.2 Equal Opportunity:

In [1015]:
show_equal_opportunity("Model 1 - Male", "Model 1 - Female", cm1[0], cm1[1])

Equal opportunity:
	Model 1 - Male: 0.7
	Model 1 - Female: 0.73
	ratio: 0.96


In [1016]:
show_equal_opportunity("Model 2 - Male", "Model 2 - Female", cm2[0], cm2[1])

Equal opportunity:
	Model 2 - Male: 0.52
	Model 2 - Female: 0.39
	ratio: 1.33


##### 2.2.3 Predictive Rate Parity

In [1017]:
show_predictive_rate_parity("Model 1 - Male", "Model 1 - Female", cm1[0], cm1[1])

Positive Predictive Parity:
	Model 1 - Male: 0.02
	Model 1 - Female: 0.17
	ratio: 0.12
Negative Predictive Parity:
	Model 1 - Male: 0.9
	Model 1 - Female: 0.77
	ratio: 1.17


In [1018]:
show_predictive_rate_parity("Model 2 - Male", "Model 2 - Female", cm2[0], cm2[1])

Positive Predictive Parity:
	Model 2 - Male: 0.34
	Model 2 - Female: 0.05
	ratio: 6.8
Negative Predictive Parity:
	Model 2 - Male: 0.19
	Model 2 - Female: 0.47
	ratio: 0.4


# Appendix

### 0. Necessary libraries

In [1019]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from IPython.display import display, Math
from math import ceil
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, recall_score, precision_score


### 1. Code for task 1

In [1020]:
task1_blue_results = np.array([
    [60, 5],  
    [20, 15]
], dtype=int)
task1_red_results = np.array([
    [0.25, 0.25],
    [0.25, 0.25]
], dtype=float)

def task1_display_results(data: np.ndarray, group: str):
    return np.array([
        [group, "Will use XAI", "Will not use XAI", "Total"],
        ["Enrolled in training", data[0,0], data[0,1], np.sum(data, axis=1)[0]],
        ["not enrolled in training", data[1,0], data[1,1], np.sum(data, axis=1)[1]],
        ["Total", np.sum(data, axis=0)[0], np.sum(data, axis=0)[1], np.sum(data)]
    ])

def _calculate_true_positive_rate(data: np.ndarray) -> float:
    return round(data[0,0] / np.sum(data, axis=0)[0], 2) # TPR=TP/(TP+FN)

def _calculate_demographic_parity(data: np.ndarray) -> float:
     return round(np.sum(data, axis=1)[1] / np.sum(data), 2) # P=TP+FP

def _calculate_positive_predictive_parity(data: np.ndarray) -> float:
    return round(data[0,0] / np.sum(data, axis=1)[0], 2) # PPV=TP/(TP+FP)
    
def _calculate_negative_predictive_parity(data: np.ndarray) -> float:
    return round(data[1,1] / np.sum(data, axis=1)[1], 2) # NPV=TN/(TN+FN)
    
def show_demographic_parity(first_group_name, second_group_name, data1=task1_blue_results, data2=task1_red_results):
    print("Demographic parity:")
    print(f"\t{first_group_name}: {_calculate_demographic_parity(data1)}")
    print(f"\t{second_group_name}: {_calculate_demographic_parity(data2)}")
    print(f"\tratio: {round(_calculate_demographic_parity(data1) / _calculate_demographic_parity(data2), 2)}")
    
def show_equal_opportunity(first_group_name, second_group_name, data1=task1_blue_results, data2=task1_red_results):
    print("Equal opportunity:")
    print(f"\t{first_group_name}: {_calculate_true_positive_rate(data1)}")
    print(f"\t{second_group_name}: {_calculate_true_positive_rate(data2)}")
    print(f"\tratio: {round(_calculate_true_positive_rate(data1) / _calculate_true_positive_rate(data2), 2)}")
    
def show_predictive_rate_parity(first_group_name, second_group_name, data1=task1_blue_results, data2=task1_red_results):    
    print("Positive Predictive Parity:")
    print(f"\t{first_group_name}: {_calculate_positive_predictive_parity(data1)}")
    print(f"\t{second_group_name}: {_calculate_positive_predictive_parity(data2)}")
    print(f"\tratio: {round(_calculate_positive_predictive_parity(data1) / _calculate_positive_predictive_parity(data2), 2)}")
    print("Negative Predictive Parity:")
    print(f"\t{first_group_name}: {_calculate_negative_predictive_parity(data1)}")
    print(f"\t{second_group_name}: {_calculate_negative_predictive_parity(data2)}")
    print(f"\tratio: {round(_calculate_negative_predictive_parity(data1) / _calculate_negative_predictive_parity(data2), 2)}")
    

### 2. Code for task 2

### 2.1. Data preparation + basic models training

In [1023]:
def check_unique_input_values():
    return df.nunique(axis=0)

def transform_data_into_numerical_values(data):
    encoder = OrdinalEncoder()
    columns_to_fit = data[["workclass", "education", "marital-status", "occupation", "relationship", "race", "gender", "native-country","income"]]
    numerical_columns = data[["age", "fnlwgt", "educational-num", "capital-gain", "capital-loss", "hours-per-week"]]
    transformed_data = encoder.fit_transform(columns_to_fit.to_numpy())
    result_y = transformed_data[:, [-1]]
    transformed_data = np.delete(transformed_data, -1, axis=1)
    result_x = np.concatenate((transformed_data, numerical_columns), axis=1)
    result_x_df = pd.DataFrame(result_x, columns = ["workclass", "education", "marital-status", "occupation", "relationship", "race", "gender", "native-country", "age", "fnlwgt", "educational-num", "capital-gain", "capital-loss", "hours-per-week"])
    return result_x_df, np.ravel(result_y)


def standardise_data(data):
    scaler = StandardScaler()
    result = scaler.fit_transform(data.to_numpy())
    return pd.DataFrame(result, columns = ["workclass", "education", "marital-status", "occupation", "relationship", "race", "gender", "native-country", "age", "fnlwgt", "educational-num", "capital-gain", "capital-loss", "hours-per-week"])
    
df = pd.read_csv("data/adult.csv")

X1, y = transform_data_into_numerical_values(df)
X2 = standardise_data(X1)

X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y, test_size=0.33, random_state=42)
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y, test_size=0.33, random_state=42)
X3_train, X3_test, y3_train, y3_test = train_test_split(X2, y, test_size=0.33, random_state=42)

model1 = LogisticRegression(max_iter=1000, class_weight=None, random_state=42, penalty="l1", solver="liblinear")
model2 = LogisticRegression(max_iter=10000, class_weight="balanced", random_state=42, penalty="l2", warm_start=True)
model1.fit(X1_train, y1_train)
model2.fit(X2_train, y2_train)
model1_pred = model1.predict(X1_test)
model2_pred = model2.predict(X2_test)

### 2.2. Fairness evaluation for basic models

Fairness metrics are calculated using functions defined above. Here we construct the classification_matrix

In [1022]:
def calculate_classification_matrix(X_test, y_test, y_pred, protected_attribute):
    data = X_test.astype(int)
    data["income"] = y_test.astype(int)
    data["prediction"] = y_pred.astype(int)
    attribute_values = data[protected_attribute].unique()
    groups_result = [0 for _ in range(0, attribute_values.shape[0])]
    for attribute_value in attribute_values:
        group_data = data[data[protected_attribute].values == attribute_value]
        P = group_data[group_data["prediction"] == 1.]
        N = group_data[group_data["prediction"] == 0.]
        groups_result[int(attribute_value)] = np.array([
            [
                P[P["income"] == P["prediction"]].shape[0], 
                N[N["income"] == N["prediction"]].shape[0]
            ],[
                P[P["income"] != P["prediction"]].shape[0],
                N[N["income"] != N["prediction"]].shape[0]
            ]
        ])
    return groups_result